In [ ]:
!pip install scikit-learn-extra

In [ ]:
!pip install -q pyyaml h5py

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import pickle
from keras import backend as K
import numpy as np
from sklearn_extra.cluster import KMedoids
from tensorflow import keras
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from datetime import datetime
from scipy.spatial import distance_matrix
import sys

In [ ]:
print(tf.__version__)
from google.colab import drive 
drive.mount('/content/drive')

2.5.0
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import datasets

In [ ]:
#dir = "/content/drive/MyDrive/deep_learning/dl_research/interpretable_RNN/hotel/"
dir = "datasets/hotel/" #directory of the datasets
with open (dir + 'y_train', 'rb') as fp:
    y_train = pickle.load(fp)

with open (dir + 'train_not_clean', 'rb') as fp:
    train_not_clean = pickle.load(fp)

with open (dir + 'test_not_clean', 'rb') as fp:
    test_not_clean = pickle.load(fp)

with open (dir + 'y_test', 'rb') as fp:
    y_test = pickle.load(fp)

# Data preprocessing

In [ ]:
def gen_sents(para):
    res = []
    for p in para:
        sents = p.split(".")
        res.append(sents)
    return res


train_noclean_sents = gen_sents(train_not_clean)
test_noclean_sents = gen_sents(test_not_clean)

In [ ]:
x_train = train_noclean_sents
y_train = [int(y) for y in y_train]

x_test = test_noclean_sents
y_test = [int(y) for y in y_test]


print('test_noclean_sents ', len(test_noclean_sents), len(test_noclean_sents[0]), test_noclean_sents[0])
print('y_train: ', y_train[:10])
print('y_test: ', y_test[:10])

test_noclean_sents  902 7 ['Share your room with roach or bug', ' Roach in the room but the staff said its just a bug not a roach, like that made it any better', ' Big hole in the bathroom wall were this bug/roach came out of', ' Wonder what other rodent will come out of the wall', 'Thank you for providing us with your feedback', ' I would like to apologize for any of the inconveniences you may have experienced during your time with us', ' I wish that I had been notified of your concerns before you had checked out and I would like to apologize on behalf of our staff']
y_train:  [1, 1, 1, 0, 1, 1, 0, 0, 1, 1]
y_test:  [0, 0, 1, 1, 0, 1, 0, 0, 0, 1]


In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print("module %s loaded" % module_url)

def embed(input):
    return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
sample_sentences = []
for p in train_noclean_sents[:2000]:
    sample_sentences.extend(p)

sample_sent_vect = embed(sample_sentences)
print(sample_sent_vect)

tf.Tensor(
[[-0.02057868 -0.02088437  0.04719457 ... -0.05737035  0.0809434
   0.03041554]
 [ 0.00906795 -0.03493962 -0.02079894 ... -0.04224585  0.0316922
   0.02062659]
 [-0.05278952  0.00524477 -0.04006641 ... -0.05303752  0.07519066
  -0.00645196]
 ...
 [ 0.0728193   0.00074287  0.02359985 ...  0.05585673  0.01407796
  -0.039259  ]
 [-0.08977903  0.03925326  0.00559623 ... -0.00292801 -0.0292188
  -0.04251549]
 [ 0.05419828 -0.03739635 -0.02377675 ... -0.02753167 -0.0237032
   0.05560818]], shape=(23294, 512), dtype=float32)


# Prototype initialization

In [ ]:
k_protos, vect_size = 10, 512
kmedoids = KMedoids(n_clusters=k_protos, random_state=0).fit(sample_sent_vect)
k_cents = kmedoids.cluster_centers_
print(k_cents.shape)

[db] k_protos:  10
(10, 512)


# Model training and testing

In [ ]:
from protoryNet import ProtoryNet

In [ ]:
pNet = ProtoryNet() 

In [ ]:
model = pNet.createModel(k_cents)

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.string, name='input_1'), name='input_1', description="created by layer 'input_1'")
l2 =  KerasTensor(type_spec=TensorSpec(shape=(None, 512), dtype=tf.float32, name=None), name='keras_layer/StatefulPartitionedCall:0', description="created by layer 'keras_layer'")
l3 =  KerasTensor(type_spec=TensorSpec(shape=(1, None, 512), dtype=tf.float32, name=None), name='tf.expand_dims/ExpandDims:0', description="created by layer 'tf.expand_dims'")
proto_layer =  <protoryNet.ProtoryNet.createModel.<locals>.prototypeLayer object at 0x7fd210a4b150>
distance_layer =  <protoryNet.ProtoryNet.createModel.<locals>.distanceLayer object at 0x7fd210c25690>
dist_hot_vect  KerasTensor(type_spec=TensorSpec(shape=(None, None, 10), dtype=tf.float32, name=None), name='distance_layer/PartitionedCall:0', description="created by layer 'distance_layer'")
z =  KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='dense/Sigmoid:0', d

In [ ]:
pNet.train(x_train,y_train,x_test,y_test)

Epoch  0
i =   0
[db] pro sent dist =  Tensor("model/proto_layer/StatefulPartitionedCall:0", shape=(None, None, 10), dtype=float32)
[db] r =  Tensor("Sum_2:0", shape=(10,), dtype=float32)
db y:  Tensor("data_1:0", shape=(None,), dtype=int64) Tensor("strided_slice:0", shape=(), dtype=int64)
y_pred  Tensor("custom_model/tf.compat.v1.squeeze/Squeeze:0", shape=(1,), dtype=float32)
y_val  Tensor("ExpandDims:0", shape=(1,), dtype=int64)
[db] pro sent dist =  Tensor("model/proto_layer/StatefulPartitionedCall:0", shape=(None, None, 10), dtype=float32)
[db] r =  Tensor("Sum_2:0", shape=(10,), dtype=float32)
db y:  Tensor("data_1:0", shape=(None,), dtype=int64) Tensor("strided_slice:0", shape=(), dtype=int64)
y_pred  Tensor("custom_model/tf.compat.v1.squeeze/Squeeze:0", shape=(1,), dtype=float32)
y_val  Tensor("ExpandDims:0", shape=(1,), dtype=int64)
1/1 [==============================] - 9s 9s/step - loss: 0.6975
Evaluating y_pred, y  [0.49114528] 0 0
Evaluating y_pred, y  [0.5002696] 1 1
Evalu

KeyboardInterrupt: ignored